# **APAN 5400 - Group 4**

## MongoDB Process

In [1]:
# import
import pymongo
import json
import pandas as pd
from pymongo import MongoClient
from pprint import pprint

#### Import data

In [2]:
# import data
import pandas as pd
df = pd.read_csv ('/content/US_counties_COVID19_health_weather_data.csv')

In [3]:
len(df)

790331

In [4]:
# Since the dataset is too large 
# Select the latest 1/3 of the data 
df2 = df.tail(int(len(df)/3))

In [5]:
len(df2)

263443

#### Save the data frame as csv file 

In [6]:
# save the data frame as csv file 
df2.to_csv ('/Users/yukesong/Downloads/US_counties_COVID19_health_weather_data_2.csv')

#### initialize MongoDB client database 

In [7]:
client = MongoClient('localhost',27017) 
db = client.apan5400
collection=db.project

#### import data from csv file 

In [8]:

def csv_to_json(filename, header=None):
    data = pd.read_csv(filename, header=header)
    return data.to_dict('records')

#### Insert file to MongoDB collection

In [ ]:
# The connection cannot be established on Colab due to the time limitation, but we can do this on Jupyter Notebook
# You can run this code through Jupyter-lab due to the limitation of Colab runtime
collection.insert_many(csv_to_json('/Users/yukesong/Downloads/US_counties_COVID19_health_weather_data_2.csv',header=0))

In [ ]:
#collection.drop()

## Spark - Making Queries

### Initiate and configure Spark

In [10]:
!pip install spark
!pip install pyspark

from pyspark.conf import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext



In [11]:
sc = SparkContext() 
config = sc.getConf()
config.set('spark.cores.max','4')
config.set('spark.executor.memory', '8G')
config.set('spark.driver.maxResultSize', '8g')
config.set('spark.kryoserializer.buffer.max', '512m')
config.set("spark.driver.cores", "4")

sc.stop()

In [12]:
sc = SparkContext(conf = config) 
sqlContext = SQLContext(sc)
print("Using Apache Spark Version", sc.version)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Using Apache Spark Version 3.2.1


### Read a large CSV file into Spark DataFrame

In [13]:
covid_file = "/Users/yukesong/Downloads/US_counties_COVID19_health_weather_data_2.csv"

In [14]:
covid_sdf = sqlContext.read.format("csv") \
                        .options(header='true', inferschema='true', treatEmptyValuesAsNulls='true') \
                        .load(covid_file)
covid_sdf.count()

263443

In [15]:
covid_sdf.columns

['_c0',
 'date',
 'county',
 'state',
 'fips',
 'cases',
 'deaths',
 'stay_at_home_announced',
 'stay_at_home_effective',
 'lat',
 'lon',
 'total_population',
 'area_sqmi',
 'population_density_per_sqmi',
 'num_deaths',
 'years_of_potential_life_lost_rate',
 'percent_fair_or_poor_health',
 'average_number_of_physically_unhealthy_days',
 'average_number_of_mentally_unhealthy_days',
 'percent_low_birthweight',
 'percent_smokers',
 'percent_adults_with_obesity',
 'food_environment_index',
 'percent_physically_inactive',
 'percent_with_access_to_exercise_opportunities',
 'percent_excessive_drinking',
 'num_alcohol_impaired_driving_deaths',
 'num_driving_deaths',
 'percent_driving_deaths_with_alcohol_involvement',
 'num_chlamydia_cases',
 'chlamydia_rate',
 'teen_birth_rate',
 'num_uninsured',
 'percent_uninsured',
 'num_primary_care_physicians',
 'primary_care_physicians_rate',
 'num_dentists',
 'dentist_rate',
 'num_mental_health_providers',
 'mental_health_provider_rate',
 'preventable_h

### Top 10 counties with the highest covid-19 risk in terms of covid-19 death rate.

In [16]:
from pyspark.sql.functions import *
# query
covid_sdf = covid_sdf.withColumn('death_rate',
                        covid_sdf.deaths/covid_sdf.cases)

death_top = covid_sdf.filter(covid_sdf.date == "2020-12-04").sort(desc("death_rate")).select("state","county","death_rate").limit(10)
# show
death_top.show()

+-------------+-----------+-------------------+
|        state|     county|         death_rate|
+-------------+-----------+-------------------+
|        Texas|     Kenedy|0.15384615384615385|
|      Montana|  Petroleum|              0.125|
|        Texas|      Foard|0.11428571428571428|
|Massachusetts|   Franklin|0.10614525139664804|
|     Nebraska|      Grant|                0.1|
|        Texas|     Marion| 0.0913978494623656|
|        Texas|      Garza|0.09090909090909091|
|     Virginia|Northampton|0.08857142857142856|
|        Texas|     Crosby|0.08791208791208792|
|        Texas|     Baylor|0.08771929824561403|
+-------------+-----------+-------------------+



In [14]:
# write 
death = death_top.withColumn("top_10_death_rate", lit("top_10_death_rate")).select("state","county","death_rate","top_10_death_rate")
death.write.format('csv').save("/Users/yukesong/Downloads/death_rate10.csv",header = 'true')

### We assume the percent vaccinated, smoker percentage, hiv prevalence rate will affect the covid-19 death rate.

#### Which are top 10 counties with lowest percent vaccinated? Is there any overlap with the Top 10 highest Covid-19 death rate?


In [17]:
# delete null values
covid_sdf_no = covid_sdf.filter(covid_sdf.percent_vaccinated.isNotNull())
# query
vaccinated_top = covid_sdf_no.filter(covid_sdf_no.date == "2020-12-04").sort(asc("percent_vaccinated")).select("state","county","percent_vaccinated").limit(10)
# show
vaccinated_top.show()

+------------+------------------+------------------+
|       state|            county|percent_vaccinated|
+------------+------------------+------------------+
|      Alaska|  Nome Census Area|               4.0|
|      Kansas|           Greeley|               5.0|
|South Dakota|           Bennett|               7.0|
|South Dakota|            Corson|               8.0|
|     Montana|           Daniels|               9.0|
|    Nebraska|             Dundy|               9.0|
|    Nebraska|              Rock|               9.0|
|       Texas|            Fisher|               9.0|
|      Alaska|Bethel Census Area|              10.0|
|    Nebraska|            Cherry|              10.0|
+------------+------------------+------------------+



In [18]:
# wirte csv 
vaccinated = vaccinated_top.withColumn("top_10_low_vaccinated", lit("Top_10_low_vaccinated")) \
  .select("state","county","percent_vaccinated","top_10_low_vaccinated")
vaccinated.show()
vaccinated.write.format('csv').save("/Users/yukesong/Downloads/vaccinated_low.csv",header = 'true')


+------------+------------------+------------------+---------------------+
|       state|            county|percent_vaccinated|top_10_low_vaccinated|
+------------+------------------+------------------+---------------------+
|      Alaska|  Nome Census Area|               4.0| Top_10_low_vaccin...|
|      Kansas|           Greeley|               5.0| Top_10_low_vaccin...|
|South Dakota|           Bennett|               7.0| Top_10_low_vaccin...|
|South Dakota|            Corson|               8.0| Top_10_low_vaccin...|
|     Montana|           Daniels|               9.0| Top_10_low_vaccin...|
|    Nebraska|             Dundy|               9.0| Top_10_low_vaccin...|
|    Nebraska|              Rock|               9.0| Top_10_low_vaccin...|
|       Texas|            Fisher|               9.0| Top_10_low_vaccin...|
|      Alaska|Bethel Census Area|              10.0| Top_10_low_vaccin...|
|    Nebraska|            Cherry|              10.0| Top_10_low_vaccin...|
+------------+-----------

#### Which are top 10 counties with highest smoker percentage ? Is there any overlap with  the Top 10 highest Covid-19 death rate?


In [19]:
# change data type
covid_sdf = covid_sdf.withColumn("percent_smokers",col("percent_smokers").cast("double"))
# query
smokers_top = covid_sdf.filter(covid_sdf.date == "2020-12-04").sort(desc("percent_smokers")).select("state","county","percent_smokers").limit(10)
# show
smokers_top.show()


+------------+--------------------+---------------+
|       state|              county|percent_smokers|
+------------+--------------------+---------------+
|South Dakota|                Todd|   41.491308954|
|      Alaska|Kusilvak Census Area|   41.000948612|
|North Dakota|               Sioux|   38.704763265|
|South Dakota|       Oglala Lakota|   37.822241285|
|North Dakota|             Rolette|   37.579005224|
|South Dakota|             Buffalo|   36.013489571|
|   Wisconsin|           Menominee|   33.579999574|
|South Dakota|              Corson|   32.135084754|
|South Dakota|             Ziebach|   32.021174804|
|    Kentucky|            McCreary|   31.877699897|
+------------+--------------------+---------------+



In [20]:

# wirte csv 
smokers = smokers_top.withColumn("top_10_smokers_rate", lit("Top_10_smokers_rate")) \
  .select("state","county","percent_smokers","top_10_smokers_rate")

smokers.write.format('csv').save("/Users/yukesong/Downloads/smokers_rate.csv",header = 'true')

#### Which are top 10 counties with highest hiv prevalence rate ? Is there any overlap with  the Top 10 highest Covid-19 death rate?


In [21]:
# calculate hiv_rate
covid_sdf = covid_sdf.withColumn('hiv_rate',
                        covid_sdf.num_hiv_cases/covid_sdf.total_population)
# query
hiv_top = covid_sdf.filter(covid_sdf.date == "2020-12-04").sort(desc("hiv_rate")).select("state","county","hiv_rate").limit(10)
# show
hiv_top.show()

+--------------------+--------------------+--------------------+
|               state|              county|            hiv_rate|
+--------------------+--------------------+--------------------+
|District of Columbia|District of Columbia|0.021995147258990395|
|             Florida|               Union|0.020054093277920706|
|            Maryland|      Baltimore city| 0.01803220611916264|
|          California|       San Francisco|0.014710413721565316|
|             Georgia|              Fulton|0.013588066098356971|
|           Louisiana|             Orleans|0.012506463457309843|
|            New York|       New York City|0.011710642485825686|
|          New Jersey|               Essex|0.011705985218007888|
|             Florida|            Hamilton|0.011210137863807269|
|             Florida|             Broward|0.010254965714837588|
+--------------------+--------------------+--------------------+



In [22]:
# wirte csv 
hiv = hiv_top.withColumn("top_10_hiv_rate", lit("Top_10_hiv")) \
  .select("state","county","hiv_rate","top_10_hiv_rate")
hiv.show()
hiv.write.format('csv').save("/Users/yukesong/Downloads/hiv.csv",header = 'true')

+--------------------+--------------------+--------------------+---------------+
|               state|              county|            hiv_rate|top_10_hiv_rate|
+--------------------+--------------------+--------------------+---------------+
|District of Columbia|District of Columbia|0.021995147258990395|     Top_10_hiv|
|             Florida|               Union|0.020054093277920706|     Top_10_hiv|
|            Maryland|      Baltimore city| 0.01803220611916264|     Top_10_hiv|
|          California|       San Francisco|0.014710413721565316|     Top_10_hiv|
|             Georgia|              Fulton|0.013588066098356971|     Top_10_hiv|
|           Louisiana|             Orleans|0.012506463457309843|     Top_10_hiv|
|            New York|       New York City|0.011710642485825686|     Top_10_hiv|
|          New Jersey|               Essex|0.011705985218007888|     Top_10_hiv|
|             Florida|            Hamilton|0.011210137863807269|     Top_10_hiv|
|             Florida|      

### As for socioeconomics, we assume that poverty rate and age will also impact the death rate of covid-19.


#### Which are top 10 counties with highest percent_below_poverty? Is there any overlap with the Top 10 highest Covid-19 death rate?


In [23]:
# change data type
covid_sdf = covid_sdf.withColumn("percent_below_poverty",col("percent_below_poverty").cast("double"))
# query
poverty_top = covid_sdf.filter(covid_sdf.date == "2020-12-04").sort(desc("percent_below_poverty")).select("state","county","percent_below_poverty").limit(10)
# show
poverty_top.show()

+------------+-------------+---------------------+
|       state|       county|percent_below_poverty|
+------------+-------------+---------------------+
|South Dakota|Oglala Lakota|                 53.9|
|   Louisiana| East Carroll|                 48.7|
|South Dakota|         Todd|                 47.1|
|South Dakota|       Corson|                 46.8|
|South Dakota|     Mellette|                 46.2|
| Mississippi|       Holmes|                 45.0|
|    Kentucky|     McCreary|                 42.5|
| Mississippi|    Humphreys|                 42.5|
|     Georgia|      Stewart|                 41.4|
| Mississippi|    Claiborne|                 41.2|
+------------+-------------+---------------------+



In [24]:
# wirte csv 
poverty = poverty_top.withColumn("top_10_poverty", lit("Top_10_poverty")) \
  .select("state","county","percent_below_poverty","top_10_poverty")
poverty.show()
poverty.write.format('csv').save("/Users/yukesong/Downloads/poverty.csv",header = 'true')

+------------+-------------+---------------------+--------------+
|       state|       county|percent_below_poverty|top_10_poverty|
+------------+-------------+---------------------+--------------+
|South Dakota|Oglala Lakota|                 53.9|Top_10_poverty|
|   Louisiana| East Carroll|                 48.7|Top_10_poverty|
|South Dakota|         Todd|                 47.1|Top_10_poverty|
|South Dakota|       Corson|                 46.8|Top_10_poverty|
|South Dakota|     Mellette|                 46.2|Top_10_poverty|
| Mississippi|       Holmes|                 45.0|Top_10_poverty|
|    Kentucky|     McCreary|                 42.5|Top_10_poverty|
| Mississippi|    Humphreys|                 42.5|Top_10_poverty|
|     Georgia|      Stewart|                 41.4|Top_10_poverty|
| Mississippi|    Claiborne|                 41.2|Top_10_poverty|
+------------+-------------+---------------------+--------------+



#### Which are top 10 counties with highest percent_age65_and_older? Is there any overlap with the Top 10 highest Covid-19 death rate?


In [25]:
# change data type
covid_sdf = covid_sdf.withColumn("percent_age_65_and_older",col("percent_age_65_and_older").cast("double"))
# query
old_top = covid_sdf.filter(covid_sdf.date == "2020-12-04").sort(desc("percent_age_65_and_older")).select("state","county","percent_age_65_and_older").limit(10)
# show
old_top.show()

+----------+---------+------------------------+
|     state|   county|percent_age_65_and_older|
+----------+---------+------------------------+
|   Florida|   Sumter|                    53.1|
|  Colorado|  Mineral|                    37.6|
|   Florida|Charlotte|                    37.6|
|  Nebraska|   Hooker|                    37.4|
|   Arizona|   La Paz|                    36.1|
|  Virginia| Highland|                    36.1|
|    Oregon|  Wheeler|                    35.8|
|   Florida|   Citrus|                    35.1|
|  Virginia|Lancaster|                    34.9|
|New Mexico|   Sierra|                    34.6|
+----------+---------+------------------------+



In [26]:
# write csv
old = old_top.withColumn("top_10_old", lit("Top_10_old")) \
  .select("state","county","percent_age_65_and_older","top_10_old")
old.show()
old.write.format('csv').save("/Users/yukesong/Downloads/old.csv",header = 'true')

+----------+---------+------------------------+----------+
|     state|   county|percent_age_65_and_older|top_10_old|
+----------+---------+------------------------+----------+
|   Florida|   Sumter|                    53.1|Top_10_old|
|  Colorado|  Mineral|                    37.6|Top_10_old|
|   Florida|Charlotte|                    37.6|Top_10_old|
|  Nebraska|   Hooker|                    37.4|Top_10_old|
|   Arizona|   La Paz|                    36.1|Top_10_old|
|  Virginia| Highland|                    36.1|Top_10_old|
|    Oregon|  Wheeler|                    35.8|Top_10_old|
|   Florida|   Citrus|                    35.1|Top_10_old|
|  Virginia|Lancaster|                    34.9|Top_10_old|
|New Mexico|   Sierra|                    34.6|Top_10_old|
+----------+---------+------------------------+----------+



## Neo4j - Interactive Process / Data Visualization


### Connect to Neo4j Desktop

In [27]:
!pip install neo4j

     |████████████████████████████████| 90 kB 3.3 MB/s 
  Created wheel for neo4j: filename=neo4j-4.4.3-py3-none-any.whl size=116069 sha256=bdca502ee0baed783dc4bdbdca81fb147c08eaac6f61ab35e39d727b9cc54a28
  Stored in directory: /root/.cache/pip/wheels/db/dd/76/acacd519878f133f2f869aec70db548d89e04013209c3c62bc
Successfully built neo4j


In [28]:
from neo4j import GraphDatabase
import csv

driver = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j","123"))
session = driver.session()

In [ ]:
# delete node and relationship
query = ("MATCH (n) DETACH DELETE n")
result = session.run(query)

### Create Nodes and Relationship

#### Top 10 counties with the highest covid-19 risk in terms of covid-19 death rate

In [ ]:
query_death_rate = (
    "LOAD CSV WITH HEADERS FROM 'file:///Users/alisa/.Neo4jDesktop/relate-data/projects/project-51e4abb6-1ddc-4c61-981c-1f79eb637cf4/US_counties_COVID19_health_weather_data.csv' AS line "
    "CREATE (percent_smoker_info:Percent_smoker_info { \
                 percent_smokers: toFloat(line.percent_smokers)\
                            } \
            )"
    
     "MERGE (county:County {name: line.county})"
     "MERGE (state:State {name: line.state})"
     "MERGE (top_10_death_rate:Top_10_death_rate {name: line.top_10_death_rate})"
    
     "CREATE (death_rate_info)-[:BELONGS_TO]->(county)"
     "CREATE (county)-[:LOCATED_IN]->(state)"
     "CREATE (state)-[:WITH_IN]->(top_10_death_rate)"
    )

result1 = session.run(query_death_rate)

#### Code for Neo4j Desktop
MATCH (n) DETACH DELETE n

LOAD CSV WITH HEADERS FROM 'http://localhost:11001/project-cc583e33-ac6d-44a4-87ae-c10aa31cdd19/new.csv' AS line 

CREATE (death_rate_info:Death_rate_info { 
                     death_rate: toInteger(line.death_rate)
                            } 
            )
    
MERGE  (county:County {name: line.county})\
MERGE  (state:State {name: line.state})\
MERGE  (top_10_death_rate:Top_10_death_rate {name: line.top_10_death_rate})

CREATE (death_rate_info)-[:BELONGS_TO]->(county)\
CREATE (county)-[:LOCATED_IN]->(state)\
CREATE (state)-[:WITH_IN]->(top_10_death_rate)

return death_rate_info,county,state,top_10_death_rate

#### TOP 10 counties with highest smoker percentage

In [ ]:
# delete node and relationship
query = ("MATCH (n) DETACH DELETE n")
result = session.run(query)

In [ ]:
query_smoke = (
    "LOAD CSV WITH HEADERS FROM 'file:///Users/alisa/.Neo4jDesktop/relate-data/projects/project-51e4abb6-1ddc-4c61-981c-1f79eb637cf4/US_counties_COVID19_health_weather_data.csv' AS line "
    "CREATE (percent_smoker_info:Percent_smoker_info { \
                 percent_smokers: toFloat(line.percent_smokers)\
                            } \
            )"
    
     "MERGE (county:County {name: line.county})"
     "MERGE (state:State {name: line.state})"
     "MERGE (top_10_smokers_rate:Top_10_smokers_rate{name: line.top_10_smokers_rate})"
    
     "CREATE (percent_smoker_info)-[:BELONGS_TO]->(county)"
     "CREATE (county)-[:LOCATED_IN]->(state)"
     "CREATE (state)-[:WITH_IN]->(top_10_smokers_rate)"
    )

result2 = session.run(query_smoke)

#### Code for Neo4j Desktop
MATCH (n) DETACH DELETE n

LOAD CSV WITH HEADERS FROM 'http://localhost:11001/project-cc583e33-ac6d-44a4-87ae-c10aa31cdd19/new.csv' AS line 

CREATE (percent_smoker_info:Percent_smoker_info { 
            percent_smokers: toFloat(line.percent_smokers)
                            } 
            )
    
MERGE  (county:County {name: line.county})\
MERGE  (state:State {name: line.state})\
MERGE  (top_10_smokers_rate:Top_10_smokers_rate{name: line.top_10_smokers_rate})

CREATE (percent_smoker_info)-[:BELONGS_TO]->(county)\
CREATE (county)-[:LOCATED_IN]->(state)\
CREATE (state)-[:WITH_IN]->(top_10_smokers_rate)

return percent_smoker_info,county,state,top_10_smokers_rate

#### TOP 10 counties with lowest percent vaccinated

In [ ]:
# delete node and relationship
query = ("MATCH (n) DETACH DELETE n")
result = session.run(query)

In [ ]:
query_vaccinated = (
    "LOAD CSV WITH HEADERS FROM 'file:///Users/alisa/.Neo4jDesktop/relate-data/projects/project-51e4abb6-1ddc-4c61-981c-1f79eb637cf4/US_counties_COVID19_health_weather_data.csv' AS line "
    "CREATE (vaccinated_info:Vaccinated_info { \
                 percent_vaccinated: toFloat(line.percent_vaccinated)\
                            } \
            )"
    
     "MERGE (county:County {name: line.county})"
     "MERGE (state:State {name: line.state})"
     "MERGE (top_10_low_vaccinated:Top_10_low_vaccinated {name: line.top_10_low_vaccinated})"
    
     "CREATE (vaccinated_info)-[:BELONGS_TO]->(county)"
     "CREATE (county)-[:LOCATED_IN]->(state)"
     "CREATE (state)-[:WITH_IN]->(top_10_low_vaccinated)"
    )

result3 = session.run(query_vaccinated)

#### Code for Neo4j Desktop
MATCH (n) DETACH DELETE n

LOAD CSV WITH HEADERS FROM 'http://localhost:11001/project-cc583e33-ac6d-44a4-87ae-c10aa31cdd19/new.csv' AS line 

CREATE (vaccinated_info:Vaccinated_info { 
            vaccinated_info: toFloat(line.vaccinated_info)
                            } 
            )
    
MERGE  (county:County {name: line.county})\
MERGE  (state:State {name: line.state})\
MERGE  (top_10_low_vaccinated:Top_10_low_vaccinated{name: line.top_10_low_vaccinated})

CREATE (vaccinated_info)-[:BELONGS_TO]->(county)\
CREATE (county)-[:LOCATED_IN]->(state)\
CREATE (state)-[:WITH_IN]->(top_10_low_vaccinated)

return vaccinated_info,county,state,top_10_low_vaccinated

#### TOP 10 counties with highest hiv prevalence rate 

In [ ]:
# delete node and relationship
query = ("MATCH (n) DETACH DELETE n")
result = session.run(query)

In [ ]:
query_hiv_rate = (
    "LOAD CSV WITH HEADERS FROM 'file:///Users/alisa/.Neo4jDesktop/relate-data/projects/project-51e4abb6-1ddc-4c61-981c-1f79eb637cf4/US_counties_COVID19_health_weather_data.csv' AS line "
    "CREATE (hiv_rate_info:Hiv_rate_info { \
                 hiv_rate: toFloat(line.hiv_rate)\
                            } \
            )"
    
     "MERGE (county:County {name: line.county})"
     "MERGE (state:State {name: line.state})"
     "MERGE (top_10_hiv:Top_10_hiv {name: line.top_10_hiv})"
    
     "CREATE (hiv_rate_info)-[:BELONGS_TO]->(county)"
     "CREATE (county)-[:LOCATED_IN]->(state)"
     "CREATE (state)-[:WITH_IN]->(top_10_hiv)"
    )

result4 = session.run(query_hiv_rate)

#### Code for Neo4j Desktop
MATCH (n) DETACH DELETE n

LOAD CSV WITH HEADERS FROM 'http://localhost:11001/project-cc583e33-ac6d-44a4-87ae-c10aa31cdd19/new.csv' AS line 

CREATE (hiv_rate_info:Hiv_rate_info { 
                 hiv_rate: toFloat(line.hiv_rate)
                            } 
            )
    
MERGE  (county:County {name: line.county})\
MERGE  (state:State {name: line.state})\
MERGE (top_10_hiv:Top_10_hiv {name: line.top_10_hiv})

CREATE (hiv_rate_info)-[:BELONGS_TO]->(county)\
CREATE (county)-[:LOCATED_IN]->(state)\
CREATE (state)-[:WITH_IN]->(top_10_hiv)

return hiv_rate_info,county,state,top_10_hiv

#### TOP 10 counties with highest percent_below_poverty

In [ ]:
# delete node and relationship
query = ("MATCH (n) DETACH DELETE n")
result = session.run(query)

In [ ]:
query_poverty = (
    "LOAD CSV WITH HEADERS FROM 'file:///Users/alisa/.Neo4jDesktop/relate-data/projects/project-51e4abb6-1ddc-4c61-981c-1f79eb637cf4/US_counties_COVID19_health_weather_data.csv' AS line "
    "CREATE (percent_below_poverty_info:Percent_below_poverty_info { \
                 percent_below_poverty: toFloat(line.percent_below_poverty)\
                            } \
            )"
    
     "MERGE (county:County {name: line.county})"
     "MERGE (state:State {name: line.state})"
     "MERGE (top_10_poverty:Top_10_poverty {name: line.top_10_poverty})"
    
     "CREATE (percent_below_poverty_info)-[:BELONGS_TO]->(county)"
     "CREATE (county)-[:LOCATED_IN]->(state)"
     "CREATE (state)-[:WITH_IN]->(top_10_poverty)"
    )

result5 = session.run(query_poverty)

#### Code for Neo4j Desktop
MATCH (n) DETACH DELETE n

LOAD CSV WITH HEADERS FROM 'http://localhost:11001/project-cc583e33-ac6d-44a4-87ae-c10aa31cdd19/new.csv' AS line 

CREATE (percent_below_poverty_info:Percent_below_poverty_info { 
            percent_below_poverty: toFloat(line.percent_below_poverty)
                            } 
            )
    
MERGE  (county:County {name: line.county})\
MERGE  (state:State {name: line.state})\
MERGE  (top_10_poverty:Top_10_poverty{name: line.top_10_poverty})

CREATE (percent_below_poverty_info)-[:BELONGS_TO]->(county)\
CREATE (county)-[:LOCATED_IN]->(state)\
CREATE (state)-[:WITH_IN]->(top_10_poverty)

return percent_below_poverty_info,county,state,top_10_poverty

#### TOP 10 counties with highest percent_age65_and_older

In [ ]:
# delete node and relationship
query = ("MATCH (n) DETACH DELETE n")
result = session.run(query)

In [ ]:
query_old = (
    "LOAD CSV WITH HEADERS FROM 'file:///Users/alisa/.Neo4jDesktop/relate-data/projects/project-51e4abb6-1ddc-4c61-981c-1f79eb637cf4/US_counties_COVID19_health_weather_data.csv' AS line "
    "CREATE (percent_age_65_and_older_info:Percent_age_65_and_older_info { \
                 percent_age_65_and_older: toFloat(line.percent_age_65_and_older)\
                            } \
            )"
    
     "MERGE (county:County {name: line.county})"
     "MERGE (state:State {name: line.state})"
     "MERGE (top_10_old:Top_10_old {name: line.top_10_old})"
    
     "CREATE (percent_age_65_and_older)-[:BELONGS_TO]->(county)"
     "CREATE (county)-[:LOCATED_IN]->(state)"
     "CREATE (state)-[:WITH_IN]->(top_10_old)"
    )

result6 = session.run(query_old)

#### Code for Neo4j Desktop
MATCH (n) DETACH DELETE n

LOAD CSV WITH HEADERS FROM 'http://localhost:11001/project-cc583e33-ac6d-44a4-87ae-c10aa31cdd19/new.csv' AS line 

CREATE (percent_age_65_and_older_info:Percent_age_65_and_older_info { 
            percent_age_65_and_older: toFloat(line.percent_age_65_and_older)
                            } 
            )
    
MERGE  (county:County {name: line.county})\
MERGE  (state:State {name: line.state})\
MERGE  (top_10_old:Top_10_old{name: line.top_10_old})

CREATE (percent_age_65_and_older_info)-[:BELONGS_TO]->(county)\
CREATE (county)-[:LOCATED_IN]->(state)\
CREATE (state)-[:WITH_IN]->(top_10_old)

return percent_age_65_and_older_info,county,state,top_10_old